In [ ]:
import json
from pathlib import Path


RESULTS_DIR = Path(
    "/workspaces/cv/data/inferences/20240815_060150-yolov9-c-20240815-maui/640x640_0.2x0.2/0.01/action-aviation-multicamera/20250405_west_coast_4camera/flight_1"
)

all_inferences = []
n_inferences = 0
min_score = 0.01
inference_dir = RESULTS_DIR / "inferences"
assert inference_dir.exists()
for fpath in sorted(inference_dir.iterdir()):
    n_inferences += 1
    if n_inferences % 1000 == 0:
        print(n_inferences, end="\r")
    with open(fpath) as f:
        inference = json.load(f)
    inference = [i for i in inference if i["score"] >= min_score]
    if inference:
        img_path = RESULTS_DIR / "visuals_with_gt" / f"{fpath.stem}.jpg"
        if not img_path.exists():
            print("No image!", img_path)
            continue
        all_inferences.append((inference, inference_dir.name, RESULTS_DIR.name, img_path))
print(f"Found {len(all_inferences)} inferences from {n_inferences} with something")

Found 3101 inferences from 14952 with something


In [3]:
# OK, copy them over and order by score
import shutil
import os

odir = Path("/workspaces/cv/.tmp/all_inferences")
if odir.exists():
    print("=> removing existing")
    shutil.rmtree(odir)
    print("=> done removing")
odir.mkdir(exist_ok=True, parents=True)
for idx, (inference, survey, flight, imgpath) in enumerate(all_inferences):
    if idx % 100 == 0:
        print(idx, end="\r")
    opath = odir / f"{survey}__{flight}__{imgpath.name}"
    shutil.copy(imgpath, opath)